# Optimization for Portfolio Allocation

Portfolio Optimisation is the process of selecting the best portfolio, or rather the best set of possible weights for a basket of assets, given some sort of objective function. Common objective functions include maximising returns, minimising risk, maximising return-risk ratio or minimising tail risk. The aim of our study is to understand deeper into how Machine Learning can be used to tackle such a complex asset allocation problem, and potentially devise a quantitative strategy that exceeds traditional methods.

The most common strategy that comes to mind when we talk about Portfolio Optimisation would be Markowitz’s Mean-Variance Optimisation. It aims to maximise portfolio returns while minimising portfolio risk, or rather the portfolio’s volatility. <br>
<br>
For example, investing in two separate negatively correlated stocks instead of a single stock is likely to reduce the risk, or in this case the volatility, of the portfolio. Such an action can also potentially generate greater returns even with the reduced risk. However, Markowitz’s model requires the forecasted portfolio return and volatility as an input to the model.<br>
<br>
Instead of a simple moving average forecast, we see an opportunity to use Machine Learning to gather a more accurate forecast of the future expected portfolio returns and volatility. With the assumption that the underlying theory of Mean-Variance Optimisation works, we would then be able to come up with an optimal portfolio. This strategy would, hence, combine more advanced Machine Learning techniques with traditional financial theory.<br>

In [1]:
# Install
! pip install yfinance
! pip install PyPortfolioOpt


# Import 
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import HRPOpt
from pypfopt.efficient_frontier import EfficientCVaR

In [2]:
# Tickers 
tickers = ["BLK", "BAC", "AAPL", "TM", "WMT",
           "JD", "INTU", "MA", "UL", "CVS",
           "DIS", "AMD", "NVDA", "PBI", "TGT"]

# Let's focus on 2001-2023 
date_b = "2001-01-01"
date_e = "2023-07-23"
start_date = dt.datetime.strptime(date_b, "%Y-%m-%d")
end_date = dt.datetime.strptime(date_e, "%Y-%m-%d")
df = data.history(start=start_date,end=end_date)

ohlc = yf.download(tickers, start = start_date, end = end_date)
print("ohlc type is :", type(ohlc))

# Drop na
prices = ohlc["Adj Close"].dropna(how="all")
prices.head()
prices.plot(figsize = (16,12))

NameError: name 'data' is not defined

In [ ]:
# Calculating the covariance matrix

sample_cov = risk_models.sample_cov(prices, frequency=252)

sample_cov.head()

In [ ]:
plotting.plot_covariance(sample_cov, plot_correlation=True)

In [ ]:
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
plotting.plot_covariance(S, plot_correlation=True)

In [ ]:
# expected returns
expected_returns.capm_return(prices)

##  I) Mean-Variance Optimization

In [ ]:

portfolio = prices
mu = expected_returns.mean_historical_return(portfolio)
S = risk_models.CovarianceShrinkage(portfolio).ledoit_wolf()

Next, let’s import the EfficientFrontier module and calculate the weights. Here, we will use the max Sharpe statistic. The Sharpe ratio is the ratio between returns and risk. The lower the risk and the higher the returns, the higher the Sharpe ratio. The algorithm looks for the maximum Sharpe ratio, which translates to the portfolio with the highest return and lowest risk. Ultimately, the higher the Sharpe ratio, the better the performance of the portfolio.

In [ ]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print(dict(cleaned_weights))
print(ef.portfolio_performance(verbose=True))

In [ ]:
latest_prices = get_latest_prices(portfolio)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Our algorithm says we should invest in 143 shares of MA and 223 shares of AAPL.


We see that our portfolio performs with an expected annual return of 32.3%.
Mean variance optimization doesn’t perform very well since it makes many simplifying assumptions, such as returns being normally distributed and the need for an invertible covariance matrix.

## II)  Hierarchical Risk Parity (HRP)

In [ ]:

returns = portfolio.pct_change().dropna()
# Run the optimization algorithm to get the weights:

hrp = HRPOpt(returns)
hrp_weights = hrp.optimize()

# Print the performance of the portfolio and the weights
hrp.portfolio_performance(verbose=True)
print(dict(hrp_weights))

# let’s calculate the discrete allocation using our weights

da_hrp = DiscreteAllocation(hrp_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_hrp.greedy_portfolio()
print("Discrete allocation (HRP):", allocation)
print("Funds remaining (HRP): ${:.2f}".format(leftover))

## III) Mean Conditional Value at Risk (mCVAR)

It works by measuring the worst-case scenarios for each asset in the portfolio, which is represented here by losing the most money. The worst-case loss for each asset is then used to calculate weights to be used for allocation for each asset.

In [ ]:
S = portfolio.cov()
ef_cvar = EfficientCVaR(mu, S)
cvar_weights = ef_cvar.min_cvar()

cleaned_weights = ef_cvar.clean_weights()
print(dict(cleaned_weights))

In [ ]:
da_cvar = DiscreteAllocation(cvar_weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da_cvar.greedy_portfolio()
print("Discrete allocation (CVAR):", allocation)
print("Funds remaining (CVAR): ${:.2f}".format(leftover))